In [2]:
#Loading essential libraries
import numpy as np
import pandas as pd

In [3]:
#Loading datasets
tr = pd.read_csv('train.csv')
camp = pd.read_csv('campaign_data.csv')
coup = pd.read_csv('coupon_item_mapping.csv')
item = pd.read_csv('item_data.csv')
cust_dem = pd.read_csv('customer_demographics.csv')
cust_trans = pd.read_csv('customer_transaction_data.csv')
ts = pd.read_csv('test_QyjYwdj.csv')

In [4]:
#Loading cutomer level & coupon level aggregation data which we prepared earlier
cust_agg = pd.read_csv('cust_agg.csv')
coup_agg = pd.read_csv('coup_agg.csv')

In [5]:
#Merging coupon and item data
coup_item = coup.merge(item, on='item_id', how='left')
coup_item.info()

#Merging cust_trans and item data
cust_item = cust_trans.merge(item, on='item_id', how='left')
cust_item.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92663 entries, 0 to 92662
Data columns (total 5 columns):
coupon_id     92663 non-null int64
item_id       92663 non-null int64
brand         92663 non-null int64
brand_type    92663 non-null object
category      92663 non-null object
dtypes: int64(3), object(2)
memory usage: 4.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1324566 entries, 0 to 1324565
Data columns (total 10 columns):
date               1324566 non-null object
customer_id        1324566 non-null int64
item_id            1324566 non-null int64
quantity           1324566 non-null int64
selling_price      1324566 non-null float64
other_discount     1324566 non-null float64
coupon_discount    1324566 non-null float64
brand              1324566 non-null int64
brand_type         1324566 non-null object
category           1324566 non-null object
dtypes: float64(3), int64(4), object(3)
memory usage: 111.2+ MB


In [7]:
#Getting id level data from training and test dataset
id_data = pd.concat([tr,ts],sort=False, axis='rows')[['campaign_id', 'coupon_id', 'customer_id']]

In [8]:
#Number of unique coupons, unique customers in each campaign?
camp_agg = id_data.groupby(['campaign_id']).agg({'coupon_id':['nunique', 'count'], 'customer_id':['nunique']})
camp_agg.columns=['camp_' + '_'.join(col).strip() for col in camp_agg.columns.values]

#frequency percentage of each campaing in train and test dataset?
camp_agg.rename(columns={'camp_coupon_id_count':'camp_frequency_perc'}, inplace=True)
camp_agg['camp_frequency_perc'] = camp_agg['camp_frequency_perc']/len(id_data)

In [9]:
#How many distinct coupons and ciustomers are present in each campaign that were not present in any other campaign?
tmp=pd.DataFrame(columns=['camp_customer_distinct_count', 'camp_coupon_distinct_count'])
for x in id_data.campaign_id.unique():
    tmp1 = len(np.setdiff1d(id_data[id_data['campaign_id']==x]['customer_id'].unique(),\
                                  id_data[id_data['campaign_id']!=x]['customer_id'].unique()))
    tmp2 = len(np.setdiff1d(id_data[id_data['campaign_id']==x]['coupon_id'].unique(),\
                                  id_data[id_data['campaign_id']!=x]['coupon_id'].unique()))
    tmp = pd.concat([tmp, pd.DataFrame({'camp_customer_distinct_count':tmp1,
                                        'camp_coupon_distinct_count':tmp2}, index=[x])])
    
camp_agg['camp_customer_distinct_count'] = camp_agg.index.map(tmp['camp_customer_distinct_count'])
camp_agg['camp_coupon_distinct_count'] = camp_agg.index.map(tmp['camp_coupon_distinct_count'])

In [10]:
#How many unique items & unique brands are present in each campaign?
tmp=pd.DataFrame(columns=['camp_item_nunique', 'camp_brand_nunique', 'camp_Local_item_nunique',
                          'camp_Established_item_nunique', 'camp_Local_brand_nunique', 'camp_Established_brand_nunique'])

for x in id_data.campaign_id.unique():
    x1 = id_data[id_data['campaign_id']==x]['coupon_id'].unique()
    tmp1 = coup_item[coup_item['coupon_id'].isin(x1)]['item_id'].nunique()
    tmp2 = coup_item[coup_item['coupon_id'].isin(x1)]['brand'].nunique()
    #How many Local & Established unique items & unique brands are present in each campaign?
    
    tmp3 = coup_item[coup_item['coupon_id'].isin(x1) & (coup_item['brand_type']=='Local')]['item_id'].nunique()
    tmp4 = coup_item[coup_item['coupon_id'].isin(x1) & (coup_item['brand_type']=='Local')]['brand'].nunique()
    
    tmp5 = coup_item[coup_item['coupon_id'].isin(x1) & (coup_item['brand_type']=='Established')]['item_id'].nunique()
    tmp6 = coup_item[coup_item['coupon_id'].isin(x1) & (coup_item['brand_type']=='Established')]['brand'].nunique()
    
    tmp = pd.concat([tmp, pd.DataFrame({'camp_item_nunique':tmp1, 'camp_brand_nunique':tmp2, 'camp_Local_item_nunique':tmp3,
                                        'camp_Established_item_nunique':tmp5, 'camp_Local_brand_nunique':tmp4,
                                        'camp_Established_brand_nunique':tmp6}, index=[x])])
for c in tmp.columns:
    camp_agg[c] = camp_agg.index.map(tmp[c])

In [11]:
#For Each 19 categories how many unique items & unique brands are present in each campaign?
for x in coup_item['category'].unique():
    tmp_cat = coup_item[(coup_item['category']==x)]
    cname1 = 'camp_'+str(x)+'_item_nunique'
    cname2 = 'camp_'+str(x)+'_brand_nunique'
    tmp = pd.DataFrame(columns=[cname1,cname2])
    for x1 in id_data['campaign_id'].unique():
        tmp_coup = id_data[id_data['campaign_id']==x1]['coupon_id']
        tmp1 = tmp_cat[tmp_cat['coupon_id'].isin(tmp_coup)]['item_id'].nunique()
        tmp2 = tmp_cat[tmp_cat['coupon_id'].isin(tmp_coup)]['brand'].nunique()
        tmp = pd.concat([tmp, pd.DataFrame({cname1:tmp1, cname2:tmp2}, index=[x1])])
    camp_agg[cname1] = camp_agg.index.map(tmp[cname1])
    camp_agg[cname2] = camp_agg.index.map(tmp[cname2])

In [12]:
#How many transactions happened for all the unique customers inside each campaign?
#How many transactions happened for all the unique coupons present inside each campaign?
camp_agg['camp_customer_trans_Count']=0
camp_agg['camp_coupon_trans_Count']=0
for x in id_data['campaign_id'].unique():
    x1 = id_data[id_data['campaign_id']==x]['customer_id'].unique()
    x2 = id_data[id_data['campaign_id']==x]['coupon_id'].unique()
    camp_agg.loc[camp_agg.index==x, 'camp_customer_trans_Count'] = cust_agg[cust_agg.index.isin(x1)]['cust_trans_count'].sum()
    camp_agg.loc[camp_agg.index==x, 'camp_coupon_trans_Count'] = coup_agg[coup_agg.index.isin(x2)]['coup_trans_count'].sum()

In [13]:
#How many transactions happened for the distinct customers & distinct coupons present in each campaign?
camp_agg['camp_customer_distinct_trans_Count']=0
camp_agg['camp_coupon_distinct_trans_Count']=0
for x in id_data['campaign_id'].unique():
    x1 = np.setdiff1d(id_data[id_data['campaign_id']==x]['customer_id'].unique(),
                      id_data[id_data['campaign_id']!=x]['customer_id'].unique())
    x2 = np.setdiff1d(id_data[id_data['campaign_id']==x]['coupon_id'].unique(),
                      id_data[id_data['campaign_id']!=x]['coupon_id'].unique())
    camp_agg.loc[camp_agg.index==x, 'camp_customer_distinct_trans_Count'] = cust_agg[cust_agg.index.isin(x1)]['cust_trans_count'].sum()
    camp_agg.loc[camp_agg.index==x, 'camp_coupon_distinct_trans_Count'] = coup_agg[coup_agg.index.isin(x2)]['coup_trans_count'].sum()

In [14]:
#Saving the aggregated data into a csv file
camp_agg.reset_index().to_csv('camp_agg.csv', index=False)

In [15]:
camp_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 1 to 30
Data columns (total 49 columns):
camp_coupon_id_nunique                       28 non-null int64
camp_frequency_perc                          28 non-null float64
camp_customer_id_nunique                     28 non-null int64
camp_customer_distinct_count                 28 non-null int64
camp_coupon_distinct_count                   28 non-null int64
camp_item_nunique                            28 non-null int64
camp_brand_nunique                           28 non-null int64
camp_Local_item_nunique                      28 non-null int64
camp_Established_item_nunique                28 non-null int64
camp_Local_brand_nunique                     28 non-null int64
camp_Established_brand_nunique               28 non-null int64
camp_Grocery_item_nunique                    28 non-null int64
camp_Grocery_brand_nunique                   28 non-null int64
camp_Bakery_item_nunique                     28 non-null int64
camp_Bakery_b